In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent
# from art.estimators.classification import PyTorchClassifier

import time

In [2]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',

        }
head = pd.DataFrame([head])
head.to_csv("./RSLAD_100.csv", mode='a', index=False)


In [3]:
def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_test))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    


        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        

        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,

        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("./RSLAD_100.csv", mode='a', index=False, header=False)



In [4]:
# def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
#     model.eval()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model.to(device)
    
#     all_preds = []
#     all_labels = []
#     probabilities = []

#     num_classes = len(np.unique(y_test))
    
#     X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
#     y_test_tensor = torch.tensor(y_test, dtype=torch.long)
#     test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
#     test_loader = DataLoader(dataset=test_dataset)

#     with torch.no_grad():
        
#         for inputs, labels in test_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             preds = torch.argmax(outputs, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
#         all_preds = np.array(all_preds)
#         all_labels = np.array(all_labels)
#         probabilities = np.array(probabilities)

#         np.save(f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Defense_Label/WUSTL_Def10/y_pred_{attack_name}{eps}_Def10.npy", all_preds)


In [5]:
x_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
x_train = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_train.npy')
x_val = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_val.npy')
y_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')
y_train = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_train.npy')
y_val = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_val.npy')

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [7]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [8]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.autograd import Variable

class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [10]:
# Initialize model, optimizer, and loss function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

# Early stopping variables
min_delta = 0.001
patience = 5
patience_counter = 0
best_loss = float('inf')


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import numpy as np

def attack_pgd(model,train_batch_data,train_batch_labels,attack_iters=10,step_size=2/255.0,epsilon=8.0/255.0):
    ce_loss = torch.nn.CrossEntropyLoss().cuda()
    train_ifgsm_data = train_batch_data.detach() + torch.zeros_like(train_batch_data).uniform_(-epsilon,epsilon)
    train_ifgsm_data = torch.clamp(train_ifgsm_data,0,1)
    for i in range(attack_iters):
        train_ifgsm_data.requires_grad_()
        logits = model(train_ifgsm_data)
        loss = ce_loss(logits,train_batch_labels.cuda())
        loss.backward()
        train_grad = train_ifgsm_data.grad.detach()
        train_ifgsm_data = train_ifgsm_data + step_size*torch.sign(train_grad)
        train_ifgsm_data = torch.clamp(train_ifgsm_data.detach(),0,1)
        train_ifgsm_pert = train_ifgsm_data - train_batch_data
        train_ifgsm_pert = torch.clamp(train_ifgsm_pert,-epsilon,epsilon)
        train_ifgsm_data = train_batch_data + train_ifgsm_pert
        train_ifgsm_data = train_ifgsm_data.detach()
    return train_ifgsm_data

def rslad_inner_loss(model,
                teacher_logits,
                x_natural,
                y,
                optimizer,
                step_size=0.003,
                epsilon=0.031,
                perturb_steps=10,
                beta=6.0):
    # define KL-loss
    criterion_kl = nn.KLDivLoss(size_average=False,reduce=False)
    model.eval()
    batch_size = len(x_natural)
    # generate adversarial example
    x_adv = x_natural.detach() + 0.001 * torch.randn(x_natural.shape).cuda().detach()

    for _ in range(perturb_steps):
        x_adv.requires_grad_()
        with torch.enable_grad():
            loss_kl = criterion_kl(F.log_softmax(model(x_adv), dim=1),
                                       F.softmax(teacher_logits, dim=1))
            loss_kl = torch.sum(loss_kl)
        grad = torch.autograd.grad(loss_kl, [x_adv])[0]
        x_adv = x_adv.detach() + step_size * torch.sign(grad.detach())
        x_adv = torch.min(torch.max(x_adv, x_natural - epsilon), x_natural + epsilon)
        x_adv = torch.clamp(x_adv, 0.0, 1.0)

    model.train()

    x_adv = Variable(torch.clamp(x_adv, 0.0, 1.0), requires_grad=False)
    # zero gradient
    optimizer.zero_grad()
    logits = model(x_adv)
    return logits
  

In [12]:
import os
import argparse
import torch
# from rslad_loss import *
# from cifar10_models import *
import torchvision
from torchvision import datasets, transforms
import time
# # we fix the random seed to 0, in the same computer, this method can make the results same as before.
# torch.manual_seed(0)
# torch.cuda.manual_seed_all(0)
# torch.backends.cudnn.deterministic = True

# prefix = 'mobilenet_v2-CIFAR10_RSLAD'
epochs = 10
batch_size = 128
epsilon = 8/255.0
multiply = 1



student = DNNModel(input_shape,output_shape )
student = torch.nn.DataParallel(student)
student = student.cuda()
student.train()
optimizer = optim.SGD(student.parameters(), lr=0.1, momentum=0.9, weight_decay=2e-4)
def kl_loss(a,b):
    loss = -a*b + torch.log(b+1e-5)*b
    return loss
teacher = DNNModel(input_shape,output_shape )
teacher.load_state_dict(torch.load('/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/dnn_pytorch.pt'))
teacher = torch.nn.DataParallel(teacher)
teacher = teacher.cuda()
teacher.eval()

/tmp/ipykernel_1165/2870078014.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  teacher.load_state_dict(torch.load('/home/jovyan/Sample_Based_Extension/WUSTL/transfer_at

DataParallel(
  (module): DNNModel(
    (fc1): Linear(in_features=42, out_features=50, bias=True)
    (fc2): Linear(in_features=50, out_features=30, bias=True)
    (fc3): Linear(in_features=30, out_features=20, bias=True)
    (fc4): Linear(in_features=20, out_features=5, bias=True)
  )
)

In [13]:


# for epoch in range(0,epochs):
#     train_loss = 0.0
#     for step,(train_batch_data,train_batch_labels) in enumerate(train_loader):
#         student.train()
#         train_batch_data = train_batch_data.float().cuda()
#         train_batch_labels = train_batch_labels.cuda()
#         optimizer.zero_grad()
#         with torch.no_grad():
#             teacher_logits = teacher(train_batch_data)

#         adv_logits = rslad_inner_loss(student,teacher_logits,train_batch_data,train_batch_labels,optimizer,step_size=2/255.0,epsilon=epsilon,perturb_steps=3)
#         student.train()
#         nat_logits = student(train_batch_data)
#         kl_Loss1 = kl_loss(torch.log(F.softmax(adv_logits,dim=1)),F.softmax(teacher_logits.detach(),dim=1))
#         kl_Loss2 = kl_loss(torch.log(F.softmax(nat_logits,dim=1)),F.softmax(teacher_logits.detach(),dim=1))
#         # multiply 10 to keep consistent with CIFAR-10 dataset
#         kl_Loss1 = multiply*torch.mean(kl_Loss1)
#         kl_Loss2 = multiply*torch.mean(kl_Loss2)
#         loss = 5.0/6.0*kl_Loss1 + 1.0/6.0*kl_Loss2
#         loss.backward()
#         optimizer.step()
#         if step % 1000 == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, step * len(train_batch_data), len(train_loader.dataset),
#                 100. * step / len(train_loader), loss.item()))
#         train_loss += loss.item()

#     avg_train_loss = train_loss / len(train_loader)

#     model.eval()
#     val_train_loss = 0.0
#     correct_predictions = 0
#     with torch.no_grad():
#         for inputs, labels in val_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
            
#             outputs = student(inputs)
#             kl_Loss1 = kl_loss(torch.log(F.softmax(adv_logits,dim=1)),F.softmax(teacher_logits.detach(),dim=1))
#             kl_Loss2 = kl_loss(torch.log(F.softmax(nat_logits,dim=1)),F.softmax(teacher_logits.detach(),dim=1))
#             # multiply 10 to keep consistent with CIFAR-10 dataset
#             kl_Loss1 = multiply*torch.mean(kl_Loss1)
#             kl_Loss2 = multiply*torch.mean(kl_Loss2)
#             loss = 5.0/6.0*kl_Loss1 + 1.0/6.0*kl_Loss2
            
#             val_train_loss += loss.item()
#             _, predicted = torch.max(outputs.data, 1)
#             correct_predictions += (predicted == labels).sum().item()

#     avg_val_loss = val_train_loss / len(val_loader)
#     val_accuracy = correct_predictions / len(val_dataset)

#     print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
#     # print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}")

    
#     # if (epoch%2 == 0):
#     #     test_accs = []
#     #     student.eval()
#     #     for step,(test_batch_data,test_batch_labels) in enumerate(test_loader):
#     #         test_ifgsm_data = attack_pgd(student,test_batch_data,test_batch_labels,attack_iters=20,step_size=0.003,epsilon=8.0/255.0)
#     #         logits = student(test_ifgsm_data)
#     #         predictions = np.argmax(logits.cpu().detach().numpy(),axis=1)
#     #         predictions = predictions - test_batch_labels.cpu().detach().numpy()
#     #         test_accs = test_accs + predictions.tolist()
#     #     test_accs = np.array(test_accs)
#     #     test_acc = np.sum(test_accs==0)/len(test_accs)
#     #     print('robust acc',np.sum(test_accs==0)/len(test_accs))
#     #     # torch.save(student.state_dict(),'./models/'+prefix+str(np.sum(test_accs==0)/len(test_accs))+'.pth')


#     # # Early stopping check using min_delta
#     # if best_loss - avg_val_loss > min_delta:
#     #     best_loss = avg_val_loss
#     #     patience_counter = 0
#     # else:
#     #     patience_counter += 1

#     # if patience_counter >= patience:
#     #     print("Early stopping triggered")
#     #     break

    

In [14]:
student.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Defense/RSLAD/RSLAD_100.pt"))

/tmp/ipykernel_1165/2601722920.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Defens

<All keys matched successfully>

In [15]:
# calculate_performance_metrics(x_test, y_test, student, 'DNN', 'baseline', '0')

In [16]:
def print_empty_file():
    new_row = {
        "model" : "0",
        "attack_model" : "0",
        'epsilon': "0",
        'Accuracy': "0",
        'Macro Precision': "0",
        'Weighted Precision': "0",
        'Macro Recall': "0",
        'Weighted Recall': "0",
        'Macro F1': "0",
        'Weighted F1': "0",
    }
    new_row_df = pd.DataFrame([new_row])
    new_row_df.to_csv("./RSLAD_100.csv", mode='a', index=False, header=False)

In [17]:
epsilon = 0
Def = "Def10"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}"
                calculate_performance_metrics(x_test_adv, y_test_adv, student, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start XGB 100 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB/WUSTL_Input100/x_test_adv_baseline_Def10.npy not found
start XGB 100 BIM
(2, 42) (2,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 FGSM
(23, 42) (23,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 PGD
(2, 42) (2,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 DF
(8852, 42) (8852,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9158382286488929

macro
Precision: 0.5952166832588677
Recall: 0.5955715018141516
F1 Score: 0.5945675854632219

weighted
Precision: 0.9521585350738536
Recall: 0.9158382286488929
F1 Score: 0.9328263883114823

start XGB 100 AutoPGD
(3, 42) (3,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB/WUSTL_Input100/x_test_adv_ZOO_Def10.npy not found
start XGB 100 CaFA
(206, 42) (206,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.470873786407767

macro
Precision: 0.23769796380090497
Recall: 0.4701834862385321
F1 Score: 0.23225005642067253

weighted
Precision: 0.49804781882880117
Recall: 0.470873786407767
F1 Score: 0.4815722414673319

start XGB 100 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB/WUSTL_Input100/x_test_adv_SINIFGSM_Def10.npy not found
start XGB 100 VNIFGSM
(3, 42) (3,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 50 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB/WUSTL_Input50/x_test_adv_baseline_Def10.npy not found
start XGB 50 BIM
(18, 42) (18,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 50 FGSM
(7, 42) (7,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 50 PGD
(18, 42) (18,)
Ac

...

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.2698191933240612

macro
Precision: 0.34241126070991434
Recall: 0.35161453986023145
F1 Score: 0.17284083316580046

weighted
Precision: 0.8616742109178565
Recall: 0.2698191933240612
F1 Score: 0.33522051388119856

start RF 10 PGD
(2, 42) (2,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 DF
(10063, 42) (10063,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classifi

Accuracy: 0.8900924177680613

macro
Precision: 0.5622362359384598
Recall: 0.5744616457403932
F1 Score: 0.5664170582701697

weighted
Precision: 0.92804917471466
Recall: 0.8900924177680613
F1 Score: 0.9069706458106824

start RF 10 AutoPGD
(1, 42) (1,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF/WUSTL_Input10/x_test_adv_ZOO_Def10.npy not found
start RF 10 CaFA
(1, 42) (1,)
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start RF 10 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF/WUSTL_Input10/x_test_adv_SINIFGSM_Def10.npy not found
start RF 10 VNIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF/WUSTL_Input10/x_test_adv_VNIFGSM_Def10.npy not found
start RF 1 baseline
/home/jovyan/Sample_Based_Extensio

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classifi

In [18]:
epsilon = 0
Def = "Def10"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ExcludeCaFA/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ExcludeCaFA/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ExcludeCaFA"
                calculate_performance_metrics(x_test_adv, y_test_adv, student, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start XGB 100 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ExcludeCaFA/WUSTL_Input100/x_test_adv_baseline_Def10.npy not found
start XGB 100 BIM
(1860, 42) (1860,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9962365591397849

macro
Precision: 0.6666666666666666
Recall: 0.665374677002584
F1 Score: 0.6660194174757281

weighted
Precision: 1.0
Recall: 0.9962365591397849
F1 Score: 0.9981146257438146

start XGB 100 FGSM
(5192, 42) (5192,)
Accuracy: 0.9978813559322034

macro
Precision: 0.8472222222222222
Recall: 0.9992799162084315
F1 Score: 0.9005404696941116

weighted
Precision: 0.9988524011299436
Recall: 0.9978813559322034
F1 Score: 0.9981951440716915

start XGB 100 PGD
(1860, 42) (1860,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9962365591397849

macro
Precision: 0.6666666666666666
Recall: 0.665374677002584
F1 Score: 0.6660194174757281

weighted
Precision: 1.0
Recall: 0.9962365591397849
F1 Score: 0.9981146257438146

start XGB 100 DF
(8661, 42) (8661,)


...

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8965102286401926

macro
Precision: 0.5619062725672374
Recall: 0.5858000891756499
F1 Score: 0.5703939137646706

weighted
Precision: 0.9371512254167376
Recall: 0.8965102286401926
F1 Score: 0.91357196351892

start RF 20 AutoPGD
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input20/x_test_adv_AutoPGD_Def10.npy not found
start RF 20 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input20/x_test_adv_ZOO_Def10.npy not found
start RF 20 CaFA
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input20/x_test_adv_CaFA_Def10.npy not found
start RF 20 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input20/x_test_adv_SINIFGSM_Def10.npy not found
start RF 20 VNIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input20/x_test_adv_VNIFGSM_Def10.npy not f

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classifi

Accuracy: 0.89387627577588

macro
Precision: 0.5599264816751125
Recall: 0.5873708642466728
F1 Score: 0.5694634706495498

weighted
Precision: 0.9357381842049858
Recall: 0.89387627577588
F1 Score: 0.9109119517273476

start RF 10 AutoPGD
(50, 42) (50,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input10/x_test_adv_ZOO_Def10.npy not found
start RF 10 CaFA
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input10/x_test_adv_CaFA_Def10.npy not found
start RF 10 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input10/x_test_adv_SINIFGSM_Def10.npy not found
start RF 10 VNIFGSM
(27, 42) (27,)
Accuracy: 0.8888888888888888

macro
Precision: 0.625
Recall: 0.9423076923076923
F1 Score: 0.6693877551020408

weighted
Precision: 0.9

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [19]:
epsilon = 0
Def = "Def10"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

total_num_attack = ["2", "3"]
for num_attack in total_num_attack:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    print(x_test_adv.shape, y_test_adv.shape)
    
                    m_per_name = f"{m_name}{p}_Exclude{num_attack}Attack"
                    calculate_performance_metrics(x_test_adv, y_test_adv, student, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()


start XGB 100 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Exclude2Attack/WUSTL_Input100/x_test_adv_baseline_Def10.npy not found
start XGB 100 BIM
(2496, 42) (2496,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9963942307692307

macro
Precision: 0.6559139784946236
Recall: 0.6654501216545011
F1 Score: 0.6605928012707166

weighted
Precision: 0.9996122828784119
Recall: 0.9963942307692307
F1 Score: 0.9979967828646914

start XGB 100 FGSM
(5662, 42) (5662,)
Accuracy: 0.9595549275874249

macro
Precision: 0.5909722222222222
Recall: 0.8333422556787297
F1 Score: 0.667390434692862

weighted
Precision: 0.9834175939008597
Recall: 0.9595549275874249
F1 Score: 0.9694537930988938

start XGB 100 PGD
(2496, 42) (2496,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9963942307692307

macro
Precision: 0.6559139784946236
Recall: 0.6654501216545011
F1 Score: 0.6605928012707166

weighted
Precision: 0.9996122828784119
Recall: 0.9963942307692307
F1 Score: 0.9979967828646914

start XGB 100 DF
(8852, 42) (8852,)


...

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8930142170399918

macro
Precision: 0.5687630768335775
Recall: 0.5821959899067904
F1 Score: 0.572908542822652

weighted
Precision: 0.9373182356152827
Recall: 0.8930142170399918
F1 Score: 0.9122940560274919

start RF 10 AutoPGD
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input10/x_test_adv_AutoPGD_Def10.npy not found
start RF 10 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input10/x_test_adv_ZOO_Def10.npy not found
start RF 10 CaFA
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input10/x_test_adv_CaFA_Def10.npy not found
start RF 10 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input10/x_test_adv_SINIFGSM_Def10.npy not found
start RF 10 VNIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input10/x_test_adv_VNIFGSM

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8283870967741935

macro
Precision: 0.5731870126532419
Recall: 0.5231226680351092
F1 Score: 0.5469931741541423

weighted
Precision: 0.9117497442874396
Recall: 0.8283870967741935
F1 Score: 0.8680498139412791

start RF 1 AutoPGD
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input1/x_test_adv_AutoPGD_Def10.npy not found
start RF 1 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input1/x_test_adv_ZOO_Def10.npy not found
start RF 1 CaFA
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input1/x_test_adv_CaFA_Def10.npy not found
start RF 1 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input1/x_test_adv_SINIFGSM_Def10.npy not found
start RF 1 VNIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input1/x_test_adv_VNIFGSM_Def10.np

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

In [20]:
epsilon = 0

Def = "Def10"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

rec_list = ["Euclidean", "cosine", "manhattan"]
for rec in rec_list:
    for attack in attack_names:
        print(f"start {attack}")
        
        x_path = f"{base_path}/Recommendation_{rec}/x_test_adv_{attack}_{Def}.npy"
        y_path = f"{base_path}/Recommendation_{rec}/y_test_adv_{attack}_{Def}.npy"
    
        try:
            x_test_adv = np.load(x_path)
            y_test_adv = np.load(y_path)
            m_per_name = f"Recommendation_{rec}"
            calculate_performance_metrics(x_test_adv, y_test_adv, student, m_per_name, attack, epsilon)
        except FileNotFoundError:
            print(x_path, "not found")
            print_empty_file()



start baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/Recommendation_Euclidean/x_test_adv_baseline_Def10.npy not found
start BIM


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8405646668050654

macro
Precision: 0.5019470179502027
Recall: 0.6781402517924509
F1 Score: 0.4862380571518281

weighted
Precision: 0.9404070449199232
Recall: 0.8405646668050654
F1 Score: 0.883629634289594

start FGSM
Accuracy: 0.8994105343221145

macro
Precision: 0.7482051880657492
Recall: 0.9112479143009171
F1 Score: 0.7724869535333965

weighted
Precision: 0.9658522866489903
Recall: 0.8994105343221145
F1 Score: 0.9233002999708023

start PGD


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8405646668050654

macro
Precision: 0.5019470179502027
Recall: 0.6781402517924509
F1 Score: 0.4862380571518281

weighted
Precision: 0.9404070449199232
Recall: 0.8405646668050654
F1 Score: 0.883629634289594

start DF


...

Accuracy: 0.46863799283154123

macro
Precision: 0.48502567185759493
Recall: 0.4023282291093587
F1 Score: 0.2927640944166263

weighted
Precision: 0.9148452357845049
Recall: 0.46863799283154123
F1 Score: 0.6053135137861732

start ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/Recommendation_manhattan/x_test_adv_ZOO_Def10.npy not found
start CaFA


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.09268292682926829

macro
Precision: 0.028215534712357722
Recall: 0.3341269841269841
F1 Score: 0.047025727851755975

weighted
Precision: 0.018831338371231052
Recall: 0.09268292682926829
F1 Score: 0.031242336658580203

start SINIFGSM


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.3935483870967742

macro
Precision: 0.43166666666666664
Recall: 0.3959537572254335
F1 Score: 0.2573000529204909

weighted
Precision: 0.8189462365591398
Recall: 0.3935483870967742
F1 Score: 0.35096990764520786

start VNIFGSM
Accuracy: 0.40625

macro
Precision: 0.6935483870967742
Recall: 0.525
F1 Score: 0.3266888150609081

weighted
Precision: 0.7701612903225806
Recall: 0.40625
F1 Score: 0.26882613510520487



In [21]:
epsilon = 0
Def = "Def10"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_Cluster/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_Cluster/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_Cluster"
                calculate_performance_metrics(x_test_adv, y_test_adv, student, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start XGB 100 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Cluster/WUSTL_Input100/x_test_adv_baseline_Def10.npy not found
start XGB 100 BIM
(2, 42) (2,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 FGSM
(23, 42) (23,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 PGD
(2, 42) (2,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 DF
(8852, 42) (8852,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9158382286488929

macro
Precision: 0.5952166832588677
Recall: 0.5955715018141516
F1 Score: 0.5945675854632219

weighted
Precision: 0.9521585350738536
Recall: 0.9158382286488929
F1 Score: 0.9328263883114823

start XGB 100 AutoPGD
(3, 42) (3,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Cluster/WUSTL_Input100/x_test_adv_ZOO_Def10.npy not found
start XGB 100 CaFA
(206, 42) (206,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.470873786407767

macro
Precision: 0.23769796380090497
Recall: 0.4701834862385321
F1 Score: 0.23225005642067253

weighted
Precision: 0.49804781882880117
Recall: 0.470873786407767
F1 Score: 0.4815722414673319

start XGB 100 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Cluster/WUSTL_Input100/x_test_adv_SINIFGSM_Def10.npy not found
start XGB 100 VNIFGSM
(3, 42) (3,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 50 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Cluster/WUSTL_Input50/x_test_adv_baseline_Def10.npy not found
start XGB 50 BIM
(10, 42) (10,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 50 FGSM
(338, 42) (338,)
Accuracy: 0.6715976331360947

macro
Precision: 0.7797619047619048
Recall: 0.8659420289855072
F1 Score: 0.751655328798186

weight

...

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(58, 42) (58,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 1 DF
(1982, 42) (1982,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8319878910191726

macro
Precision: 0.3333333333333333
Recall: 0.27732929700639086
F1 Score: 0.3027632424492794

weighted
Precision: 1.0
Recall: 0.8319878910191726
F1 Score: 0.9082897273478381

start RF 1 AutoPGD
(70, 42) (70,)
Accuracy: 0.9714285714285714

macro
Precision: 0.6666666666666666
Recall: 0.9855072463768115
F1 Score: 0.7426470588235294

weighted
Precision: 0.9904761904761904
Recall: 0.9714285714285714
F1 Score: 0.9783613445378151

start RF 1 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Cluster/WUSTL_Input1/x_test_adv_ZOO_Def10.npy not found
start RF 1 CaFA
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Cluster/WUSTL_Input1/x_test_adv_CaFA_Def10.npy not found
start RF 1 SINIFGSM
(59, 42) (59,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 1 VNIFGSM
(15, 42) (15,)
Accuracy: 0.2

macro
Precision: 0.5
Recall: 0.1
F1 Score: 0.16666

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [22]:
epsilon = 0
Def = "Def10"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ActiveLearning/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ActiveLearning/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ActiveLearning"
                calculate_performance_metrics(x_test_adv, y_test_adv, student, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start XGB 100 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning/WUSTL_Input100/x_test_adv_baseline_Def10.npy not found
start XGB 100 BIM
(2, 42) (2,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 FGSM
(23, 42) (23,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 PGD
(2, 42) (2,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 DF
(8852, 42) (8852,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9158382286488929

macro
Precision: 0.5952166832588677
Recall: 0.5955715018141516
F1 Score: 0.5945675854632219

weighted
Precision: 0.9521585350738536
Recall: 0.9158382286488929
F1 Score: 0.9328263883114823

start XGB 100 AutoPGD
(3, 42) (3,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning/WUSTL_Input100/x_test_adv_ZOO_Def10.npy not found
start XGB 100 CaFA
(206, 42) (206,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.470873786407767

macro
Precision: 0.23769796380090497
Recall: 0.4701834862385321
F1 Score: 0.23225005642067253

weighted
Precision: 0.49804781882880117
Recall: 0.470873786407767
F1 Score: 0.4815722414673319

start XGB 100 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning/WUSTL_Input100/x_test_adv_SINIFGSM_Def10.npy not found
start XGB 100 VNIFGSM
(3, 42) (3,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 50 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning/WUSTL_Input50/x_test_adv_baseline_Def10.npy not found
start XGB 50 BIM
(14, 42) (14,)
Accuracy: 0.6428571428571429

macro
Precision: 0.6428571428571428
Recall: 0.7916666666666667
F1 Score: 0.5906432748538011

weighted
Precision: 0.8979591836734694
Recall: 0.6428571428571429
F1 Score: 0.6950710108604845

start XGB 50 FGSM
(313, 42) (313,)
Accuracy:

...

Accuracy: 0.8755775577557756

macro
Precision: 0.4806538250644788
Recall: 0.44450806690339223
F1 Score: 0.4612428227563319

weighted
Precision: 0.9610973019277517
Recall: 0.8755775577557756
F1 Score: 0.9151262884137766

start RF 10 AutoPGD
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning/WUSTL_Input10/x_test_adv_AutoPGD_Def10.npy not found
start RF 10 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning/WUSTL_Input10/x_test_adv_ZOO_Def10.npy not found
start RF 10 CaFA
(65, 42) (65,)
Accuracy: 0.9846153846153847

macro
Precision: 0.5
Recall: 0.49230769230769234
F1 Score: 0.49612403100775193

weighted
Precision: 1.0
Recall: 0.9846153846153847
F1 Score: 0.9922480620155039

start RF 10 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning/WUSTL_Input10/x_test_adv_SINIFGSM_Def10.npy not found
start RF 10 VNIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynami

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(149, 42) (149,)
Accuracy: 0.8322147651006712

macro
Precision: 0.5
Recall: 0.4161073825503356
F1 Score: 0.4542124542124542

weighted
Precision: 1.0
Recall: 0.8322147651006712
F1 Score: 0.9084249084249084

start RF 1 AutoPGD
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning/WUSTL_Input1/x_test_adv_AutoPGD_Def10.npy not found
start RF 1 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning/WUSTL_Input1/x_test_adv_ZOO_Def10.npy not found
start RF 1 CaFA


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(18, 42) (18,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 1 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning/WUSTL_Input1/x_test_adv_SINIFGSM_Def10.npy not found
start RF 1 VNIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning/WUSTL_Input1/x_test_adv_VNIFGSM_Def10.npy not found


In [23]:
epsilon = 0

Def = "Def10"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]
active_learning_name = ["DensityWeighted", "BatchMode"]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for ac_name in active_learning_name:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_ActiveLearning_{ac_name}/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_ActiveLearning_{ac_name}/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    m_per_name = f"{m_name}{p}_ActiveLearning_{ac_name}"
                    calculate_performance_metrics(x_test_adv, y_test_adv, student, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()



start XGB 100 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning_DensityWeighted/WUSTL_Input100/x_test_adv_baseline_Def10.npy not found
start XGB 100 BIM
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 FGSM
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 PGD
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 DF


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9158382286488929

macro
Precision: 0.5952166832588677
Recall: 0.5955715018141516
F1 Score: 0.5945675854632219

weighted
Precision: 0.9521585350738536
Recall: 0.9158382286488929
F1 Score: 0.9328263883114823

start XGB 100 AutoPGD
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning_DensityWeighted/WUSTL_Input100/x_test_adv_ZOO_Def10.npy not found
start XGB 100 CaFA


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.470873786407767

macro
Precision: 0.23769796380090497
Recall: 0.4701834862385321
F1 Score: 0.23225005642067253

weighted
Precision: 0.49804781882880117
Recall: 0.470873786407767
F1 Score: 0.4815722414673319

start XGB 100 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning_DensityWeighted/WUSTL_Input100/x_test_adv_SINIFGSM_Def10.npy not found
start XGB 100 VNIFGSM
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 50 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning_DensityWeighted/WUSTL_Input50/x_test_adv_baseline_Def10.npy not found
start XGB 50 BIM


...

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9188612099644128

macro
Precision: 0.5919995413268956
Recall: 0.5964295209454799
F1 Score: 0.5933513367002059

weighted
Precision: 0.9513446136920914
Recall: 0.9188612099644128
F1 Score: 0.9340175740941151

start RF 1 AutoPGD


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classifi

Accuracy: 0.7457627118644068

macro
Precision: 0.35077294090607797
Recall: 0.5715488215488215
F1 Score: 0.31742007238727193

weighted
Precision: 0.9856969842661265
Recall: 0.7457627118644068
F1 Score: 0.8410532897612396

start RF 1 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning_BatchMode/WUSTL_Input1/x_test_adv_ZOO_Def10.npy not found
start RF 1 CaFA
Accuracy: 0.5454545454545454

macro
Precision: 0.3333333333333333
Recall: 0.2222222222222222
F1 Score: 0.26666666666666666

weighted
Precision: 0.8181818181818182
Recall: 0.5454545454545454
F1 Score: 0.6545454545454545

start RF 1 SINIFGSM
Accuracy: 0.47099447513812154

macro
Precision: 0.42226487523992323
Recall: 0.4484330484330485
F1 Score: 0.3115179751774415

weighted
Precision: 0.8592326698550387
Recall: 0.47099447513812154
F1 Score: 0.49565392317099966

start RF 1 VNIFGSM
Accuracy: 0.972972972972973

macro
Precision: 0.8888888888888888
Recall: 0.9166666666666666
F1 Score: 0.88571428571428

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
epsilon = 0

Def = "Def10"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF"]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"
total_num_attack = ["EOAL", "EOAL_ExcludeCaFA", "EOAL_Exclude2Attack", "EOAL_Exclude3Attack"]
for num_attack in total_num_attack:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_ActiveLearning_{num_attack}/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_ActiveLearning_{num_attack}/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    m_per_name = f"{m_name}{p}_ActiveLearning_{num_attack}"
                    calculate_performance_metrics(x_test_adv, y_test_adv, student, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()



In [15]:

# torch.save(student.state_dict(), "./RSLAD_100.pt")

In [17]:
# import time

# epsilon_values = [0.01, 0.1, 0.2, 0.3]

# start_time = time.time()

# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, student, 'DNN', 'BIM', epsilon)

# end_time = time.time()
# result = end_time - start_time
# print(f"Execution Time: {result:.6f} seconds")